In [3]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load the trained model
model = YOLO("best (1).pt")

# Tic-Tac-Toe board initialization
cap = cv2.VideoCapture(0)
board = np.zeros((3, 3), dtype=int)
playing = True

# Define mapping from YOLO detection to board (adjust based on your setup)
def map_detection_to_board(x_center, y_center):
    i = int(y_center // y_step)  # Row index
    j = int(x_center // x_step)  # Column index
    return i, j

# Drawing the board
def draw_board(frame):
    for i in range(1, 3):
        cv2.line(frame, (i * x_step, 0), (i * x_step, h), (0, 0, 0), 6)  # Vertical lines
        cv2.line(frame, (0, i * y_step), (w, i * y_step), (0, 0, 0), 6)  # Horizontal lines

# Drawing X and O on the board
def draw_XO(frame, board):
    for i in range(3):
        for j in range(3):
            if board[i, j] == 1:
                cv2.putText(frame, 'X', (j * x_step + 90, i * y_step + 210), cv2.FONT_HERSHEY_DUPLEX, 6, (0, 0, 0), 15)
            elif board[i, j] == 2:
                cv2.putText(frame, 'O', (j * x_step + 90, i * y_step + 210), cv2.FONT_HERSHEY_DUPLEX, 6, (0, 0, 0), 15)

# Placing X or O based on detection (0 for X, 1 for O)
def place_symbol(class_id, i, j):
    if class_id == 0:
        board[i, j] = 1  # 'X'
    elif class_id == 1:
        board[i, j] = 2  # 'O'

# Check for the winner (1 for X, 2 for O, -1 for draw, None for ongoing)
def check_winner(board):
    for i in range(3):
        # Check rows and columns
        if np.all(board[i, :] == board[i, 0]) and board[i, 0] != 0:
            return board[i, 0]
        if np.all(board[:, i] == board[0, i]) and board[0, i] != 0:
            return board[0, i]
    # Check diagonals
    if board[0, 0] == board[1, 1] == board[2, 2] != 0:
        return board[0, 0]
    if board[0, 2] == board[1, 1] == board[2, 0] != 0:
        return board[0, 2]
    # Check for draw
    if np.all(board != 0):
        return -1
    return None

# Display winner message
def display_winner(frame, winner):
    if winner == 1:
        cv2.putText(frame, 'X WINS!', (w // 2 - 125, 60), cv2.FONT_HERSHEY_DUPLEX, 2, (0, 0, 0), 3)
    elif winner == 2:
        cv2.putText(frame, 'O WINS!', (w // 2 - 125, 60), cv2.FONT_HERSHEY_DUPLEX, 2, (0, 0, 0), 3)
    else:
        cv2.putText(frame, "IT'S A DRAW!", (w // 2 - 200, 60), cv2.FONT_HERSHEY_DUPLEX, 2, (0, 0, 0), 3)
    
    cv2.putText(frame, "To restart (r)", (w // 2 - 110, 100), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 0), 2)
    cv2.putText(frame, "To quit (q)", (w // 2 - 95, 140), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 0), 2)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print("Failed to capture video frame.")
        break

    frame = cv2.resize(frame, (900, 900))
    frame = cv2.flip(frame, 1)

    global h, w, x_step, y_step
    h, w, _ = frame.shape
    x_step = w // 3
    y_step = h // 3

    # Use YOLOv8 to detect 'X' and 'O'
    results = model.predict(source=frame, show=False)  # Disable 'show' for drawing manually
    
    for result in results:
        for box in result.boxes:
            x_center, y_center = box.xywh[0:2]  # Get center of bounding box
            class_id = int(box.cls)  # 0 for 'X', 1 for 'O'
            i, j = map_detection_to_board(x_center, y_center)
            place_symbol(class_id, i, j)

    winner = check_winner(board)
    if winner is not None:
        playing = False
        display_winner(frame, winner)

    if playing:
        draw_board(frame)
        draw_XO(frame, board)
        cv2.imshow('Tic-Tac-Toe', frame)

    key = cv2.waitKey(1)
    if key & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 640x640 1 0, 76.8ms
Speed: 4.0ms preprocess, 76.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


ValueError: not enough values to unpack (expected 2, got 1)